In [1]:
import sqlite3 
import pandas as pd
import numpy as np
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import time
import requests
import re
from tqdm import tqdm
import json

In [2]:
# --- 공통 설정 ---
REQUEST_URL = 'https://bff-page.kakao.com/graphql'
HEADERS = {
    'Content-Type': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://page.kakao.com/', 
}

# --- 저장소 ---
main_webtoon_list = []      
all_series_ids = set()      

# 요일 코드
DAY_CODES = range(1, 8) 
DAY_NAMES = {1: '월', 2: '화', 3: '수', 4: '목', 5: '금', 6: '토', 7: '일'}

# ⚠️ 중요: 아까 찾아주신 '원본 쿼리' 전체입니다. 절대 줄이지 마세요!
FULL_QUERY_STRING = """
query staticLandingDayOfWeekLayout($queryInput: StaticLandingDayOfWeekParamInput!) {
  staticLandingDayOfWeekLayout(input: $queryInput) {
    ...Layout
  }
}

fragment Layout on Layout {
  id
  type
  sections {
    ...Section
  }
  screenUid
}

fragment Section on Section {
  id
  uid
  type
  title
  ... on RecommendSection {
    isRecommendArea
    isRecommendedItems
  }
  ... on DependOnLoggedInSection {
    loggedInTitle
    loggedInScheme
  }
  ... on SchemeSection {
    scheme
  }
  ... on MetaInfoTypeSection {
    metaInfoType
  }
  ... on TabSection {
    sectionMainTabList {
      uid
      title
      isSelected
      scheme
      additionalString
      subTabList {
        uid
        title
        isSelected
        groupId
      }
    }
  }
  ... on ThemeKeywordSection {
    themeKeywordList {
      uid
      title
      scheme
    }
  }
  ... on StaticLandingDayOfWeekSection {
    isEnd
    totalCount
    param {
      categoryUid
      businessModel {
        name
        param
      }
      subcategory {
        name
        param
      }
      dayTab {
        name
        param
      }
      page
      size
      screenUid
    }
    businessModelList {
      name
      param
    }
    subcategoryList {
      name
      param
    }
    dayTabList {
      name
      param
    }
    promotionBanner {
      ...PromotionBannerItem
    }
  }
  ... on StaticLandingTodayNewSection {
    totalCount
    param {
      categoryUid
      subcategory {
        name
        param
      }
      screenUid
    }
    categoryTabList {
      name
      param
    }
    subcategoryList {
      name
      param
    }
    promotionBanner {
      ...PromotionBannerItem
    }
    viewType
  }
  ... on StaticLandingTodayUpSection {
    isEnd
    totalCount
    param {
      categoryUid
      subcategory {
        name
        param
      }
      page
    }
    categoryTabList {
      name
      param
    }
    subcategoryList {
      name
      param
    }
  }
  ... on StaticLandingRankingSection {
    isEnd
    rankingTime
    totalCount
    param {
      categoryUid
      subcategory {
        name
        param
      }
      rankingType {
        name
        param
      }
      page
      screenUid
    }
    categoryTabList {
      name
      param
    }
    subcategoryList {
      name
      param
    }
    rankingTypeList {
      name
      param
    }
    displayAd {
      ...DisplayAd
    }
    promotionBanner {
      ...PromotionBannerItem
    }
    withOperationArea
    viewType
  }
  ... on StaticLandingGenreSection {
    isEnd
    totalCount
    param {
      categoryUid
      subcategory {
        name
        param
      }
      sortType {
        name
        param
      }
      page
      isComplete
      screenUid
    }
    subcategoryList {
      name
      param
    }
    sortTypeList {
      name
      param
    }
    displayAd {
      ...DisplayAd
    }
    promotionBanner {
      ...PromotionBannerItem
    }
  }
  ... on StaticLandingFreeSeriesSection {
    isEnd
    totalCount
    param {
      categoryUid
      tab {
        name
        param
      }
      page
      screenUid
    }
    tabList {
      name
      param
    }
    promotionBanner {
      ...PromotionBannerItem
    }
  }
  ... on StaticLandingEventSection {
    isEnd
    totalCount
    param {
      categoryUid
      page
    }
    categoryTabList {
      name
      param
    }
  }
  ... on StaticLandingOriginalSection {
    isEnd
    totalCount
    originalCount
    param {
      categoryUid
      subcategory {
        name
        param
      }
      sortType {
        name
        param
      }
      isComplete
      page
      screenUid
    }
    subcategoryList {
      name
      param
    }
    sortTypeList {
      name
      param
    }
    recommendItemList {
      ...Item
    }
  }
  ... on HelixThemeSection {
    subtitle
    isRecommendArea
  }
  groups {
    ...Group
  }
}

fragment PromotionBannerItem on PromotionBannerItem {
  title
  scheme
  leftImage
  rightImage
  eventLog {
    ...EventLogFragment
  }
}

fragment EventLogFragment on EventLog {
  fromGraphql
  click {
    layer1
    layer2
    setnum
    ordnum
    copy
    imp_id
    imp_provider
  }
  eventMeta {
    id
    name
    subcategory
    category
    series
    provider
    series_id
    type
  }
  viewimp_contents {
    type
    name
    id
    imp_area_ordnum
    imp_id
    imp_provider
    imp_type
    layer1
    layer2
  }
  customProps {
    landing_path
    view_type
    helix_id
    helix_yn
    helix_seed
    content_cnt
    event_series_id
    event_ticket_type
    play_url
    banner_uid
  }
}

fragment DisplayAd on DisplayAd {
  sectionUid
  bannerUid
  treviUid
  momentUid
}

fragment Item on Item {
  id
  type
  ...BannerItem
  ...OnAirItem
  ...CardViewItem
  ...CleanViewItem
  ... on DisplayAdItem {
    displayAd {
      ...DisplayAd
    }
  }
  ...PosterViewItem
  ...StrategyViewItem
  ...RankingListViewItem
  ...NormalListViewItem
  ...MoreItem
  ...EventBannerItem
  ...PromotionBannerItem
  ...LineBannerItem
}

fragment BannerItem on BannerItem {
  bannerType
  bannerViewType
  thumbnail
  videoUrl
  badgeList
  statusBadge
  titleImage
  title
  altText
  metaList
  caption
  scheme
  seriesId
  eventLog {
    ...EventLogFragment
  }
  discountRate
  discountRateText
  backgroundColor
  characterImage
}

fragment OnAirItem on OnAirItem {
  thumbnail
  videoUrl
  titleImage
  title
  subtitleList
  caption
  scheme
}

fragment CardViewItem on CardViewItem {
  title
  altText
  thumbnail
  scheme
  badgeList
  ageGradeBadge
  statusBadge
  ageGrade
  selfCensorship
  subtitleList
  caption
  rank
  rankVariation
  isEventBanner
  categoryType
  discountRate
  discountRateText
  backgroundColor
  isBook
  isLegacy
  cardCover {
    ...CardCoverFragment
  }
  eventLog {
    ...EventLogFragment
  }
}

fragment CardCoverFragment on CardCover {
  coverImg
  coverRestricted
}

fragment CleanViewItem on CleanViewItem {
  id
  type
  showPlayerIcon
  scheme
  title
  thumbnail
  badgeList
  ageGradeBadge
  statusBadge
  subtitleList
  rank
  ageGrade
  selfCensorship
  eventLog {
    ...EventLogFragment
  }
  discountRate
  discountRateText
}

fragment PosterViewItem on PosterViewItem {
  id
  type
  showPlayerIcon
  scheme
  title
  altText
  thumbnail
  badgeList
  labelBadgeList
  ageGradeBadge
  statusBadge
  subtitleList
  rank
  rankVariation
  ageGrade
  selfCensorship
  eventLog {
    ...EventLogFragment
  }
  seriesId
  showDimmedThumbnail
  discountRate
  discountRateText
}

fragment StrategyViewItem on StrategyViewItem {
  id
  title
  count
  scheme
}

fragment RankingListViewItem on RankingListViewItem {
  title
  thumbnail
  badgeList
  ageGradeBadge
  statusBadge
  ageGrade
  selfCensorship
  metaList
  descriptionList
  scheme
  rank
  eventLog {
    ...EventLogFragment
  }
  discountRate
  discountRateText
}

fragment NormalListViewItem on NormalListViewItem {
  id
  type
  altText
  ticketUid
  thumbnail
  badgeList
  ageGradeBadge
  statusBadge
  ageGrade
  isAlaramOn
  row1
  row2
  row3 {
    id
    metaList
  }
  row4
  row5
  scheme
  continueScheme
  nextProductScheme
  continueData {
    ...ContinueInfoFragment
  }
  seriesId
  isCheckMode
  isChecked
  isReceived
  isHelixGift
  price
  discountPrice
  discountRate
  discountRateText
  showPlayerIcon
  rank
  isSingle
  singleSlideType
  ageGrade
  selfCensorship
  eventLog {
    ...EventLogFragment
  }
  giftEventLog {
    ...EventLogFragment
  }
}

fragment ContinueInfoFragment on ContinueInfo {
  title
  isFree
  productId
  lastReadProductId
  scheme
  continueProductType
  hasNewSingle
  hasUnreadSingle
}

fragment MoreItem on MoreItem {
  id
  scheme
  title
}

fragment EventBannerItem on EventBannerItem {
  bannerType
  thumbnail
  videoUrl
  titleImage
  title
  subtitleList
  caption
  scheme
  eventLog {
    ...EventLogFragment
  }
}

fragment LineBannerItem on LineBannerItem {
  title
  scheme
  subTitle
  bgColor
  rightImage
  eventLog {
    ...EventLogFragment
  }
}

fragment Group on Group {
  id
  ... on ListViewGroup {
    meta {
      title
      count
    }
  }
  ... on CardViewGroup {
    meta {
      title
      count
    }
  }
  ... on PosterViewGroup {
    meta {
      title
      count
    }
  }
  type
  dataKey
  groups {
    ...GroupInGroup
  }
  items {
    ...Item
  }
}

fragment GroupInGroup on Group {
  id
  type
  dataKey
  items {
    ...Item
  }
  ... on ListViewGroup {
    meta {
      title
      count
    }
  }
  ... on CardViewGroup {
    meta {
      title
      count
    }
  }
  ... on PosterViewGroup {
    meta {
      title
      count
    }
  }
}
"""

def crawl_all_webtoons_pagination():
    print("🚀 전체 웹툰 목록(페이지네이션 포함) 수집 시작...")

    for day_code in DAY_CODES:
        current_page = 0
        is_end = False
        collected_count = 0 
        
        pbar = tqdm(desc=f"[{DAY_NAMES[day_code]}요일] 수집 중", unit="page")
        
        while not is_end:
            try:
                # 쿼리 전체 구성
                payload = {
                    "query": FULL_QUERY_STRING, # 위에서 정의한 긴 문자열
                    "variables": {
                        "queryInput": {
                            "categoryUid": 10,
                            "dayTabUid": str(day_code),
                            "type": "Layout",
                            "screenUid": 52,
                            "page": current_page, # 페이지 변수 추가
                            "size": 30
                        }
                    }
                }
                
                response = requests.post(
                    REQUEST_URL, 
                    headers=HEADERS, 
                    data=json.dumps(payload)
                )
                response.raise_for_status()
                data = response.json()

                sections = data.get('data', {}).get('staticLandingDayOfWeekLayout', {}).get('sections', [])
                
                if not sections:
                    break
                
                main_section = sections[0]
                
                # 종료 조건 확인
                is_end = main_section.get('isEnd', True)
                
                # 아이템 추출
                items_found_in_page = 0
                for group in main_section.get('groups', []):
                    for item in group.get('items', []):
                        if item.get('type') == 'CardView':
                            series_id = item.get('eventLog', {}).get('eventMeta', {}).get('series_id')
                            if series_id:
                                main_webtoon_list.append({
                                    'series_id': int(series_id),
                                    'title': item.get('title'),
                                    'url': f'https://page.kakao.com/content/{series_id}',
                                    'category': item.get('eventLog', {}).get('eventMeta', {}).get('subcategory'),
                                    'views': item.get('subtitleList', [None])[0],
                                    # 'thumbnail_url': item.get('thumbnail'),
                                    'weekday': DAY_NAMES[day_code]
                                })
                                all_series_ids.add(int(series_id))
                                items_found_in_page += 1
                
                collected_count += items_found_in_page
                pbar.update(1)
                pbar.set_postfix({'누적': collected_count})
                
                current_page += 1
                time.sleep(0.3) # 0.3초 대기 (서버 예의 지키기)
                
            except Exception as e:
                print(f"\nError on {DAY_NAMES[day_code]} page {current_page}: {e}")
                # 에러 발생 시 반복문 탈출
                break
        
        pbar.close()

    print(f"\n✅ 수집 완료!")
    print(f"총 웹툰 개수 (중복 포함 요일별 합산): {len(main_webtoon_list)}")
    print(f"고유 작품 ID 개수: {len(all_series_ids)}")

# --- 실행 ---
# if __name__ == "__main__":
#     crawl_all_webtoons_pagination()

In [ ]:
# # 1. 요청 URL
# REQUEST_URL = 'https://bff-page.kakao.com/graphql'

# # 2. 헤더 설정
# headers = {
#     'Content-Type': 'application/json',
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
#     'Referer': 'https://page.kakao.com/', 
# }

# # 3. GraphQL 쿼리와 변수 (제공해주신 문자열 그대로 사용)
# query_body = {
#     "query":"\n    query contentHomeInfo($seriesId: Long!) {\n  contentHomeInfo(seriesId: $seriesId) {\n    about {\n      id\n      themeKeywordList {\n        uid\n        title\n        scheme\n      }\n      description\n      screenshotList\n      authorList {\n        id\n        name\n        role\n        roleDisplayName\n      }\n      detail {\n        id\n        publisherName\n        retailPrice\n        ageGrade\n        category\n        rank\n      }\n      guideTitle\n      characterList {\n        thumbnail\n        name\n        description\n      }\n      detailInfoList {\n        title\n        info\n      }\n    }\n    recommend {\n      id\n      seriesId\n      list {\n        ...ContentRecommendGroup\n      }\n    }\n  }\n}\n    \n    fragment ContentRecommendGroup on ContentRecommendGroup {\n  id\n  impLabel\n  type\n  title\n  description\n  items {\n    id\n    type\n    ...PosterViewItem\n  }\n}\n    \n\n    fragment PosterViewItem on PosterViewItem {\n  id\n  type\n  showPlayerIcon\n  scheme\n  title\n  altText\n  thumbnail\n  badgeList\n  labelBadgeList\n  ageGradeBadge\n  statusBadge\n  subtitleList\n  rank\n  rankVariation\n  ageGrade\n  selfCensorship\n  eventLog {\n    ...EventLogFragment\n  }\n  seriesId\n  showDimmedThumbnail\n  discountRate\n  discountRateText\n}\n    \n\n    fragment EventLogFragment on EventLog {\n  fromGraphql\n  click {\n    layer1\n    layer2\n    setnum\n    ordnum\n    copy\n    imp_id\n    imp_provider\n  }\n  eventMeta {\n    id\n    name\n    subcategory\n    category\n    series\n    provider\n    series_id\n    type\n  }\n  viewimp_contents {\n    type\n    name\n    id\n    imp_area_ordnum\n    imp_id\n    imp_provider\n    imp_type\n    layer1\n    layer2\n  }\n  customProps {\n    landing_path\n    view_type\n    helix_id\n    helix_yn\n    helix_seed\n    content_cnt\n    event_series_id\n    event_ticket_type\n    play_url\n    banner_uid\n  }\n}\n    ",
#     "variables":{"seriesId":60910969}} # <-- 작품 ID: 60910969

In [3]:
# --- 공통 설정 ---
REQUEST_URL = 'https://bff-page.kakao.com/graphql'
HEADERS = {
    'Content-Type': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Referer': 'https://page.kakao.com/', 
}

# --- 최종 저장될 리스트 (DF 역할을 할 딕셔너리 리스트) ---
author_role_list = []       # 작가/그림 작가/원작 작가 정보
keyword_list = []           # 키워드 정보 저장용 리스트

# 요일 코드: 1=월, 2=화, ..., 7=일 (dayTabUid)
DAY_CODES = range(1, 8) 
DAY_NAMES = {1: '월', 2: '화', 3: '수', 4: '목', 5: '금', 6: '토', 7: '일'}

# --- 상세 정보 크롤링 함수 (줄거리, 작가 정보 수집) ---
def crawl_detail_info():
    """수집된 모든 series_id에 대해 상세 정보를 크롤링합니다."""
    print("\n🚀 2단계: 상세 정보 크롤링 시작...")

    detail_query = {
        "query":"\n    query contentHomeInfo($seriesId: Long!) {\n  contentHomeInfo(seriesId: $seriesId) {\n    about {\n      id\n      themeKeywordList {\n        uid\n        title\n        scheme\n      }\n      description\n      screenshotList\n      authorList {\n        id\n        name\n        role\n        roleDisplayName\n      }\n      detail {\n        id\n        publisherName\n        retailPrice\n        ageGrade\n        category\n        rank\n      }\n      guideTitle\n      characterList {\n        thumbnail\n        name\n        description\n      }\n      detailInfoList {\n        title\n        info\n      }\n    }\n    recommend {\n      id\n      seriesId\n      list {\n        ...ContentRecommendGroup\n      }\n    }\n  }\n}\n    \n    fragment ContentRecommendGroup on ContentRecommendGroup {\n  id\n  impLabel\n  type\n  title\n  description\n  items {\n    id\n    type\n    ...PosterViewItem\n  }\n}\n    \n\n    fragment PosterViewItem on PosterViewItem {\n  id\n  type\n  showPlayerIcon\n  scheme\n  title\n  altText\n  thumbnail\n  badgeList\n  labelBadgeList\n  ageGradeBadge\n  statusBadge\n  subtitleList\n  rank\n  rankVariation\n  ageGrade\n  selfCensorship\n  eventLog {\n    ...EventLogFragment\n  }\n  seriesId\n  showDimmedThumbnail\n  discountRate\n  discountRateText\n}\n    \n\n    fragment EventLogFragment on EventLog {\n  fromGraphql\n  click {\n    layer1\n    layer2\n    setnum\n    ordnum\n    copy\n    imp_id\n    imp_provider\n  }\n  eventMeta {\n    id\n    name\n    subcategory\n    category\n    series\n    provider\n    series_id\n    type\n  }\n  viewimp_contents {\n    type\n    name\n    id\n    imp_area_ordnum\n    imp_id\n    imp_provider\n    imp_type\n    layer1\n    layer2\n  }\n  customProps {\n    landing_path\n    view_type\n    helix_id\n    helix_yn\n    helix_seed\n    content_cnt\n    event_series_id\n    event_ticket_type\n    play_url\n    banner_uid\n  }\n}\n    ",
        # 실제 상세 쿼리 (제공해주신 것)의 핵심만 남김
        "variables": {"seriesId": 0} # 여기에 series_id 삽입
    }

    # 상세 정보 저장을 위한 임시 리스트
    temp_detail_list = [] 
    
    # 중복 ID는 한 번만 처리
    for series_id in tqdm(list(all_series_ids), desc="상세 정보 수집 중"):
        try:
            detail_query["variables"]["seriesId"] = series_id
            
            response = requests.post(
                REQUEST_URL, 
                headers=HEADERS, 
                data=json.dumps(detail_query)
            )
            response.raise_for_status()
            data = response.json()
            
            about_data = data.get('data', {}).get('contentHomeInfo', {}).get('about', {})

            if about_data:
                # A. 줄거리 등 메인 정보 (나중에 Webtoon_DF에 병합)
                temp_detail_list.append({
                    'series_id': series_id,
                    'description': about_data.get('description', '설명 없음').replace('\\n', '\n'),
                    'publisher': about_data.get('detail', {}).get('publisherName'),
                    'age_limit': about_data.get('detail', {}).get('ageGrade'),
                })
                
                # B. 작가 역할별 정보 (Author_DF 생성)
                for author in about_data.get('authorList', []):
                    author_role_list.append({
                        'series_id': series_id,
                        'author_id': author.get('id', 'N/A'),
                        'name': author.get('name'),
                        'role': author.get('role'),             # Writer, Painter, Original 등
                        'role_display': author.get('roleDisplayName')
                    })
            
                # C. 키워드 정보 저장
                for keyword in about_data.get('themeKeywordList', []):
                    keyword_list.append({
                        'series_id': series_id,
                        'keyword_title': keyword.get('title'),
                    })
            
            # 짧은 지연 시간을 두어 서버 부하를 줄임
            time.sleep(0.1) 
            
        except Exception as e:
            print(f"[{series_id}] 상세 크롤링 오류 발생: {e}")
            time.sleep(1) # 에러 시 지연 시간을 늘려 임시 차단 방지

    return temp_detail_list

# --- 실행 부분 ---
if __name__ == "__main__":
    
    # 1단계 실행: 요일별 목록 수집
    crawl_all_webtoons_pagination()
    
    # 2단계 실행: 상세 정보 수집
    detail_data_list = crawl_detail_info()
    
    # --- 3. 데이터프레임 구조에 맞게 최종 정리 ---
    
    # 1. Webtoon_DF: 메인 목록 + 상세 정보(줄거리 등) 병합
    final_webtoon_df = []
    description_map = {item['series_id']: item for item in detail_data_list}
    
    for webtoon in main_webtoon_list:
        series_id = webtoon['series_id']
        detail = description_map.get(series_id, {})
        
        # 상세 정보 필드를 메인 웹툰 정보에 추가
        webtoon['description'] = detail.get('description')
        webtoon['publisher'] = detail.get('publisher')
        webtoon['age_limit_detail'] = detail.get('age_limit')
        final_webtoon_df.append(webtoon)

    # 2. Author_DF: 작가 정보
    final_author_df = author_role_list
    
    print("\n--- ✅ 최종 결과 요약 ---")
    print(f"**Webtoon_DF (메인 목록 + 줄거리)**: 총 {len(final_webtoon_df)}개 레코드")
    print(f"**Author_DF (작가 역할 정보)**: 총 {len(final_author_df)}개 레코드")

    # 예시 출력
    if final_webtoon_df:
        print("\n[Webtoon_DF 예시 (첫 번째 작품)]")
        print(f"제목: {final_webtoon_df[0]['title']}")
        print(f"요일: {final_webtoon_df[0]['weekday']}")
        print(f"줄거리: {final_webtoon_df[0]['description'][:50]}...")
        
    if final_author_df:
        print("\n[Author_DF 예시 (첫 번째 작가)]")
        print(f"작품 ID: {final_author_df[0]['series_id']}")
        print(f"이름: {final_author_df[0]['name']} ({final_author_df[0]['role_display']})")
        
    if keyword_list:
        print(f"\n**Keyword_DF (키워드 정보)**: 총 {len(keyword_list)}개 레코드")
        print("\n[Keyword_DF 예시 (첫 번째 키워드)]")
        print(f"작품 ID: {keyword_list[0]['series_id']}")
        print(f"키워드: {keyword_list[0]['keyword_title']}")

🚀 전체 웹툰 목록(페이지네이션 포함) 수집 시작...


[월요일] 수집 중: 9page [00:03,  2.25page/s, 누적=201]
[화요일] 수집 중: 10page [00:04,  2.39page/s, 누적=231]
[수요일] 수집 중: 9page [00:03,  2.26page/s, 누적=223]
[목요일] 수집 중: 9page [00:03,  2.25page/s, 누적=217]
[금요일] 수집 중: 11page [00:04,  2.28page/s, 누적=251]
[토요일] 수집 중: 10page [00:04,  2.19page/s, 누적=237]
[일요일] 수집 중: 8page [00:03,  2.28page/s, 누적=193]



✅ 수집 완료!
총 웹툰 개수 (중복 포함 요일별 합산): 1553
고유 작품 ID 개수: 1497

🚀 2단계: 상세 정보 크롤링 시작...


상세 정보 수집 중: 100%|██████████| 1497/1497 [06:31<00:00,  3.82it/s]


--- ✅ 최종 결과 요약 ---
**Webtoon_DF (메인 목록 + 줄거리)**: 총 1553개 레코드
**Author_DF (작가 역할 정보)**: 총 3981개 레코드

[Webtoon_DF 예시 (첫 번째 작품)]
제목: 회귀한 악녀는 순결서약 합니다
요일: 월
줄거리: 뼈대 있는 후작 집안, 펑펑 써도 남을 재물, 반반한 얼굴까지 가진 리디아 에반시.
그녀가...

[Author_DF 예시 (첫 번째 작가)]
작품 ID: 56901633
이름: 그렉 러카 (글)

**Keyword_DF (키워드 정보)**: 총 14788개 레코드

[Keyword_DF 예시 (첫 번째 키워드)]
작품 ID: 66813956
키워드: 현대로맨스


In [10]:
# --- 최종 저장될 리스트 (DF 역할을 할 딕셔너리 리스트) ---
thumbnail_list = []      # 웹툰 메인 정보 + 줄거리 (나중에 병합됨)

# --- 썸네일 이미지 크롤링 함수 ---
def crawl_thumbnail_info():
    """수집된 모든 series_id에 대해 썸네일 이미지 크롤링합니다."""
    print("\n🚀 2단계: 썸네일 이미지 크롤링 시작...")

    detail_query = {
        "query":"\n    query contentHomeOverview($seriesId: Long!) {\n  contentHomeOverview(seriesId: $seriesId) {\n    id\n    seriesId\n    displayAd {\n      ...DisplayAd\n      ...DisplayAd\n    }\n    content {\n      ...SeriesFragment\n    }\n    displayAd {\n      ...DisplayAd\n    }\n    lastNoticeDate\n    setList {\n      ...NormalListViewItem\n    }\n    relatedSeries {\n      ...SeriesFragment\n    }\n  }\n}\n    \n    fragment DisplayAd on DisplayAd {\n  sectionUid\n  bannerUid\n  treviUid\n  momentUid\n}\n    \n\n    fragment SeriesFragment on Series {\n  id\n  seriesId\n  title\n  thumbnail\n  landThumbnail\n  categoryUid\n  lang\n  category\n  categoryType\n  subcategoryUid\n  subcategory\n  badge\n  isAllFree\n  isWaitfree\n  ageGrade\n  state\n  onIssue\n  authors\n  description\n  pubPeriod\n  freeSlideCount\n  lastSlideAddedDate\n  waitfreeBlockCount\n  waitfreePeriodByMinute\n  bm\n  saleState\n  startSaleDt\n  saleMethod\n  discountRate\n  discountRateText\n  serviceProperty {\n    ...ServicePropertyFragment\n  }\n  operatorProperty {\n    ...OperatorPropertyFragment\n  }\n  assetProperty {\n    ...AssetPropertyFragment\n  }\n  translateProperty {\n    ...TranslatePropertyFragment\n  }\n}\n    \n\n    fragment ServicePropertyFragment on ServiceProperty {\n  viewCount\n  readCount\n  ratingCount\n  ratingSum\n  commentCount\n  pageContinue {\n    ...ContinueInfoFragment\n  }\n  todayGift {\n    ...TodayGift\n  }\n  preview {\n    ...PreviewFragment\n    ...PreviewFragment\n  }\n  waitfreeTicket {\n    ...WaitfreeTicketFragment\n  }\n  isAlarmOn\n  isLikeOn\n  ticketCount\n  purchasedDate\n  lastViewInfo {\n    ...LastViewInfoFragment\n  }\n  purchaseInfo {\n    ...PurchaseInfoFragment\n  }\n  preview {\n    ...PreviewFragment\n  }\n  ticketInfo {\n    price\n    discountPrice\n    ticketType\n  }\n}\n    \n\n    fragment ContinueInfoFragment on ContinueInfo {\n  title\n  isFree\n  productId\n  lastReadProductId\n  scheme\n  continueProductType\n  hasNewSingle\n  hasUnreadSingle\n}\n    \n\n    fragment TodayGift on TodayGift {\n  id\n  uid\n  ticketType\n  ticketKind\n  ticketCount\n  ticketExpireAt\n  ticketExpiredText\n  isReceived\n  seriesId\n}\n    \n\n    fragment PreviewFragment on Preview {\n  item {\n    ...PreviewSingleFragment\n  }\n  nextItem {\n    ...PreviewSingleFragment\n  }\n  usingScroll\n}\n    \n\n    fragment PreviewSingleFragment on Single {\n  id\n  productId\n  seriesId\n  title\n  thumbnail\n  badge\n  isFree\n  ageGrade\n  state\n  slideType\n  lastReleasedDate\n  size\n  pageCount\n  isHidden\n  remainText\n  isWaitfreeBlocked\n  saleState\n  operatorProperty {\n    ...OperatorPropertyFragment\n  }\n  assetProperty {\n    ...AssetPropertyFragment\n  }\n}\n    \n\n    fragment OperatorPropertyFragment on OperatorProperty {\n  thumbnail\n  copy\n  helixImpId\n  isTextViewer\n  selfCensorship\n  isBook\n  cashInfo {\n    discountRate\n    setDiscountRate\n  }\n  ticketInfo {\n    price\n    discountPrice\n    ticketType\n  }\n}\n    \n\n    fragment AssetPropertyFragment on AssetProperty {\n  bannerImage\n  cardImage\n  cardTextImage\n  cleanImage\n  ipxVideo\n  bannerSet {\n    ...BannerSetFragment\n  }\n  cardSet {\n    ...CardSetFragment\n  }\n  cardCover {\n    ...CardCoverFragment\n  }\n}\n    \n\n    fragment BannerSetFragment on BannerSet {\n  backgroundImage\n  backgroundColor\n  mainImage\n  titleImage\n}\n    \n\n    fragment CardSetFragment on CardSet {\n  backgroundColor\n  backgroundImage\n}\n    \n\n    fragment CardCoverFragment on CardCover {\n  coverImg\n  coverRestricted\n}\n    \n\n    fragment WaitfreeTicketFragment on WaitfreeTicket {\n  chargedPeriod\n  chargedCount\n  chargedAt\n}\n    \n\n    fragment LastViewInfoFragment on LastViewInfo {\n  isDone\n  lastViewDate\n  rate\n  spineIndex\n}\n    \n\n    fragment PurchaseInfoFragment on PurchaseInfo {\n  purchaseType\n  rentExpireDate\n  expired\n}\n    \n\n    fragment TranslatePropertyFragment on TranslateProperty {\n  category {\n    ...LocaleMapFragment\n  }\n  sub_category {\n    ...LocaleMapFragment\n  }\n}\n    \n\n    fragment LocaleMapFragment on LocaleMap {\n  ko\n  en\n  th\n}\n    \n\n    fragment NormalListViewItem on NormalListViewItem {\n  id\n  type\n  altText\n  ticketUid\n  thumbnail\n  badgeList\n  ageGradeBadge\n  statusBadge\n  ageGrade\n  isAlaramOn\n  row1\n  row2\n  row3 {\n    id\n    metaList\n  }\n  row4\n  row5\n  scheme\n  continueScheme\n  nextProductScheme\n  continueData {\n    ...ContinueInfoFragment\n  }\n  seriesId\n  isCheckMode\n  isChecked\n  isReceived\n  isHelixGift\n  price\n  discountPrice\n  discountRate\n  discountRateText\n  showPlayerIcon\n  rank\n  isSingle\n  singleSlideType\n  ageGrade\n  selfCensorship\n  eventLog {\n    ...EventLogFragment\n  }\n  giftEventLog {\n    ...EventLogFragment\n  }\n}\n    \n\n    fragment EventLogFragment on EventLog {\n  fromGraphql\n  click {\n    layer1\n    layer2\n    setnum\n    ordnum\n    copy\n    imp_id\n    imp_provider\n  }\n  eventMeta {\n    id\n    name\n    subcategory\n    category\n    series\n    provider\n    series_id\n    type\n  }\n  viewimp_contents {\n    type\n    name\n    id\n    imp_area_ordnum\n    imp_id\n    imp_provider\n    imp_type\n    layer1\n    layer2\n  }\n  customProps {\n    landing_path\n    view_type\n    helix_id\n    helix_yn\n    helix_seed\n    content_cnt\n    event_series_id\n    event_ticket_type\n    play_url\n    banner_uid\n  }\n}\n    ",
        "variables":{"seriesId":0}
    }
    
        # 썸네일 이미지 저장을 위한 임시 리스트
    temp_detail_list = [] 
    
    # 중복 ID는 한 번만 처리
    for series_id in tqdm(list(all_series_ids), desc="썸네일 이미지 수집 중"):
        try:
            detail_query["variables"]["seriesId"] = series_id
            
            response = requests.post(
                REQUEST_URL, 
                headers=HEADERS, 
                data=json.dumps(detail_query)
            )
            response.raise_for_status()
            data = response.json()
            
            thumbnail = data.get('data', {}).get('contentHomeOverview', {}).get('content', {}).get("thumbnail")
            
            if thumbnail:
                # A. 줄거리 등 메인 정보 (나중에 Webtoon_DF에 병합)
                temp_detail_list.append({
                    'series_id': series_id,
                    'thumbnail': thumbnail,
                })
            
            # 짧은 지연 시간을 두어 서버 부하를 줄임
            time.sleep(0.1) 
            
        except Exception as e:
            print(f"[{series_id}] 상세 크롤링 오류 발생: {e}")
            time.sleep(1) # 에러 시 지연 시간을 늘려 임시 차단 방지

    return temp_detail_list

# --- 실행 부분 ---
if __name__ == "__main__":
    # 3단계 실행: 상세 정보 수집
    detail_data_list = crawl_thumbnail_info()
    print(detail_data_list)
    # --- 3. 데이터프레임 구조에 맞게 최종 정리 ---
    
    # 1. Webtoon_DF: 메인 목록 + 썸네일 url 병합
    description_map = {item['series_id']: item for item in detail_data_list}
    
    for webtoon in main_webtoon_list:
        series_id = webtoon['series_id']
        detail = description_map.get(series_id, {})
        
        # 상세 정보 필드를 메인 웹툰 정보에 추가
        webtoon['thumbnailUrl'] = detail.get('thumbnail')
        final_webtoon_df.append(webtoon)


🚀 2단계: 썸네일 이미지 크롤링 시작...


썸네일 이미지 수집 중: 100%|██████████| 1497/1497 [05:23<00:00,  4.62it/s]

[{'series_id': 56901633, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=cS2c9y/hyZ8xa8AiV/q6p2AjVzpIOkeDmgkP2rF1&filename=o1'}, {'series_id': 56901634, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=Vw5S3/hyZ8xWu42b/IqJmFoScjg7nKJOCjfv5q0&filename=o1'}, {'series_id': 66813956, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=bYiT6E/hAMB6jna4I/fUZZ5cYPOKIXPlclq1x1U0&filename=o1'}, {'series_id': 67141639, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=uISDn/hAN90PjZPr/dK82f6x6uqv4kcZ2bKk741&filename=o1'}, {'series_id': 67477518, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=bejGJy/dJMcaj8l12I/WjunyepOHLbTA1H1QUjQq0&filename=o1'}, {'series_id': 51396640, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=WiTPL/hyksMAw1hV/qqTK8oy7rk9kgfR7VGKZz0&filename=o1'}, {'series_id': 60907554, 'thumbnail': '//page-images.kakaoentcdn.com/download/resource?kid=nhRcV/dJMb89SkkHi/Y98Ie9eJTmBm5EQXE

In [11]:
from collections import defaultdict

by_series = defaultdict(list)
for item in keyword_list:
    sid = item["series_id"]
    kw  = item["keyword_title"]
    by_series[sid].append(kw)

rows = []
for sid, kws in by_series.items():
    rows.append({
        "series_id": sid,
        "keywords": ", ".join(kws),   # 리스트 그대로 두고 싶으면 kws 로
    })
    
df_keywords = pd.DataFrame(rows).sort_values("series_id").reset_index(drop=True)
df_keywords.head()

,series_id,keywords
0,47196143,"게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력, 성장,..."
1,47497773,"퓨전, 먼치킨, 초능력"
2,47936014,"천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러"
3,48475105,"정통, 무림, 전쟁, 곤륜, 무당, 정파, 성장, 천재, 도사, 마교"
4,48683376,"판타지물, 신령, 초월적존재, 친구, 학생, 동물, 이야기중심"


In [12]:
# (가정) 이전에 성공적으로 수집된 author_role_list 사용
Author_DF_Raw = pd.DataFrame(author_role_list)

def split_and_clean_authors_final_v3(df, role_key):
    """
    논리 순서를 완벽하게 수정한 버전입니다.
    1. 분리 -> 2. 기존 컬럼 삭제 -> 3. 컬럼명 변경
    """
    # 1. 역할 필터링
    filtered_df = df[df['role'] == role_key].copy()
    
    # 2. 이름 분리 (임시 컬럼 'name_temp' 생성)
    filtered_df['name_temp'] = filtered_df['name'].str.split(', ')
    
    # 3. 행 분리 (Explode)
    exploded_df = filtered_df.explode('name_temp').reset_index(drop=True)
    
    # 4. [중요] 불필요한 기존 컬럼 먼저 삭제
    # - 'name': 콤마로 뭉쳐있던 원본 이름
    # - 'author_id': UUID 에러가 났던 식별자
    exploded_df = exploded_df.drop(columns=['name', 'author_id'])
    
    # 5. [중요] 임시 컬럼의 이름을 최종 이름('name')으로 변경
    final_df = exploded_df.rename(columns={'name_temp': 'name'})
    
    # 6. 컬럼 순서 정렬
    return final_df[['series_id', 'name', 'role', 'role_display']]

# --- 실행 ---

print("--- ✅ 작가 정보 분리 및 정제 (V3) ---")

# 1. 글 작가
Writer_DF = split_and_clean_authors_final_v3(Author_DF_Raw, 'writer')
print(f"\n[글 작가] 총 {len(Writer_DF)}명")
if not Writer_DF.empty:
    print(Writer_DF.head())

# 2. 그림 작가
Painter_DF = split_and_clean_authors_final_v3(Author_DF_Raw, 'illustrator')
print(f"\n[그림 작가] 총 {len(Painter_DF)}명")
if not Painter_DF.empty:
    print(Painter_DF.head())

# 3. 원작 작가
Original_Author_DF = split_and_clean_authors_final_v3(Author_DF_Raw, 'original_author')
print(f"\n[원작 작가] 총 {len(Original_Author_DF)}명")
if not Original_Author_DF.empty:
    print(Original_Author_DF.head())

--- ✅ 작가 정보 분리 및 정제 (V3) ---

[글 작가] 총 1499명
   series_id       name    role role_display
0   56901633      그렉 러카  writer            글
1   56901634  피터 J. 토마시  writer            글
2   66813956        덤벙이  writer            글
3   66813956         플랑  writer            글
4   67141639         설기  writer            글

[그림 작가] 총 1644명
   series_id     name         role role_display
0   56901633    리암 샤프  illustrator           그림
1   56901634  패트릭 글리슨  illustrator           그림
2   66813956      덤벙이  illustrator           그림
3   67141639       이매  illustrator           그림
4   67477518      함덕이  illustrator           그림

[원작 작가] 총 1148명
   series_id name             role role_display
0   66813956   이노  original_author           원작
1   67141639  문정민  original_author           원작
2   60907554   시야  original_author           원작
3   64897059  강서울  original_author           원작
4   60792873   유성  original_author           원작


In [21]:
Writer_DF.to_csv("kp_writer.csv", index=False)
Painter_DF.to_csv("kp_painter.csv", index=False)
Original_Author_DF.to_csv("kp_original.csv", index=False)

In [13]:
kp_df = pd.DataFrame(final_webtoon_df)
kp_df.head()

,series_id,title,url,category,views,weekday,description,publisher,age_limit_detail,thumbnailUrl
0,68059016,회귀한 악녀는 순결서약 합니다,https://page.kakao.com/content/68059016,로판,99.9만,월,"뼈대 있는 후작 집안, 펑펑 써도 남을 재물, 반반한 얼굴까지 가진 리디아 에반시....",NHN,All,//page-images.kakaoentcdn.com/download/resourc...
1,61822163,악당들에게 키워지는 중입니다,https://page.kakao.com/content/61822163,로판,"4,436.4만",월,즐겨보던 로판 소설의 엑스트라 소녀로 빙의한 지 1년 만에 내가 곧 쫓겨날 신세라는...,테라핀,All,//page-images.kakaoentcdn.com/download/resourc...
2,63818432,랭커를 위한 바른 생활 안내서,https://page.kakao.com/content/63818432,로판,"1,745.7만",월,"21세기, 세상의 장르는 '판타지'\n온 국민들에게 충격과 공포를 안겨준 '악몽의 ...",연담X크랙,All,//page-images.kakaoentcdn.com/download/resourc...
3,61614855,철혈검가 사냥개의 회귀,https://page.kakao.com/content/61614855,판타지,"4,255.8만",월,바스커빌 가문의 사냥개 '비키르'.\n\n사냥개로서 가문에 충성한 보답은 \n모함과...,redicestudio,All,//page-images.kakaoentcdn.com/download/resourc...
4,56566288,이번 생은 가주가 되겠습니다,https://page.kakao.com/content/56566288,로판,2억,월,"“…한번 해 보자. 내가 가주가 되는 거야.”\n\n제국에서 제일가는 가문, 롬바르...",디앤씨미디어 | 디앤씨웹툰,All,//page-images.kakaoentcdn.com/download/resourc...


In [14]:
df_merged = (
    kp_df.groupby(
        ["series_id", "title", "url", "category", "views",
         "thumbnailUrl", "description", "publisher", "age_limit_detail"],
        as_index=False
    )["weekday"]
    .agg(lambda s: ", ".join(sorted(set(s))))
)

len(df_merged)

1497

In [15]:
kp_final_df = df_merged.merge(df_keywords, on="series_id", how="left")

def combine_cat(row):
    base = str(row["category"]).strip()           # 예: "로판"
    kws  = str(row["keywords"]).strip()
    if not kws or kws == "nan":
        return base
    return f"{base}, {kws}"

kp_final_df["category"] = kp_final_df.apply(combine_cat, axis=1)

# 필요하면 keywords 컬럼은 드롭
kp_final_df = kp_final_df.drop(columns=["keywords"])
kp_final_df.head()

,series_id,title,url,category,views,thumbnailUrl,description,publisher,age_limit_detail,weekday
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,드라마,"3,388.5만",//page-images.kakaoentcdn.com/download/resourc...,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,snowcat,All,금
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,"판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",3.4억,//page-images.kakaoentcdn.com/download/resourc...,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...",인타임,All,화
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,"액션, 퓨전, 먼치킨, 초능력","4,191.7만",//page-images.kakaoentcdn.com/download/resourc...,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,카카오웹툰스튜디오,All,화
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,판타지,"3,434.9만",//page-images.kakaoentcdn.com/download/resourc...,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,다온크리에이티브,Fifteen,수
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러","4,985.7만",//page-images.kakaoentcdn.com/download/resourc...,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,다온크리에이티브,All,토


In [16]:
wirters_pivot = (
    Writer_DF
    .groupby(["series_id", "role"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

kp_final_df = kp_final_df.merge(wirters_pivot, on="series_id", how="left")

In [17]:
painters_pivot = (
    Painter_DF
    .groupby(["series_id", "role"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

kp_final_df = kp_final_df.merge(painters_pivot, on="series_id", how="left")

In [18]:
original_pivot = (
    Original_Author_DF
    .groupby(["series_id", "role"])["name"]
    .agg(lambda s: ", ".join(sorted(set(s))))
    .unstack(fill_value="")         # role을 컬럼으로 피벗
    .reset_index()
)

kp_final_df = kp_final_df.merge(original_pivot, on="series_id", how="left")

In [19]:
kp_final_df

,series_id,title,url,category,views,thumbnailUrl,description,publisher,age_limit_detail,weekday,writer,illustrator,original_author
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,드라마,"3,388.5만",//page-images.kakaoentcdn.com/download/resourc...,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,snowcat,All,금,스노우캣,스노우캣,NaN
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,"판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",3.4억,//page-images.kakaoentcdn.com/download/resourc...,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...",인타임,All,화,"김준형, 이도경","김태형, 박정열, 신C",남희성
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,"액션, 퓨전, 먼치킨, 초능력","4,191.7만",//page-images.kakaoentcdn.com/download/resourc...,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,카카오웹툰스튜디오,All,화,네스티캣,네스티캣,NaN
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,판타지,"3,434.9만",//page-images.kakaoentcdn.com/download/resourc...,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,다온크리에이티브,Fifteen,수,프리,서락,NaN
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러","4,985.7만",//page-images.kakaoentcdn.com/download/resourc...,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,다온크리에이티브,All,토,Sariel,Sariel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1492,68144406,이리초파랑,https://page.kakao.com/content/68144406,"무협, 정통, 생존, 복수, 무림, 정파, 협객",1.6만,//page-images.kakaoentcdn.com/download/resourc...,"""보국구대문(保國九大門)의 영웅들이여---!!”\n\n멸국의 위기에 의기로 일어선\...",미스터블루,Fifteen,금,사마달,사마달,NaN
1493,68148725,붉은 달밤의 끝자락에서,https://page.kakao.com/content/68148725,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 운명...",12만,//page-images.kakaoentcdn.com/download/resourc...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",연담 | 필담,Fifteen,토,오공시,오공시,NaN
1494,68148783,붉은 달밤의 끝자락에서 [19세 완전판],https://page.kakao.com/content/68148783,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 질투...",4만,//page-images.kakaoentcdn.com/download/resourc...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",연담 | 필담,Nineteen,토,오공시,오공시,NaN
1495,68187457,드래곤을 유괴하다,https://page.kakao.com/content/68187457,"판타지, 퓨전, 회귀, 먼치킨, 육아, 성장, 이종족",25.6만,//page-images.kakaoentcdn.com/download/resourc...,최강의 힘을 가진 회귀자 유지태.\n그만한 힘을 가지고 거듭 회귀를 했음에도 지구의...,MSTORYHUB8125,Fifteen,목,최진형,주황새,유주


In [20]:
kp_final_df2 = kp_final_df.drop(["views", "publisher"], axis=1)

In [21]:
kp_final_df2.to_csv("kakao_page_webtoons.csv", index=False)

In [40]:
kp_final_df2

,series_id,title,url,category,thumbnail_url,description,age_limit_detail,weekday,writer,illustrator,original_author
0,46609100,은동스+옹동스,https://page.kakao.com/content/46609100,드라마,//dn-img-page.kakao.com/download/resource?kid=...,오빠와 이별했지만 귀엽고 순수한 작은 고양이 은동은 오늘도 엄마와 함께 밝고 씩씩하...,All,금,스노우캣,스노우캣,NaN
1,47196143,달빛조각사,https://page.kakao.com/content/47196143,"판타지, 게임, 전문직, 퓨전, 전쟁, 생존, 깽판, 먼치킨, 시스템, 천재, 노력...",//dn-img-page.kakao.com/download/resource?kid=...,"아르바이트를 전전하며 어렵게 살던 소년가장 이현, 그가 ‘위드’가 되어 가상현실게임...",All,화,"김준형, 이도경","김태형, 박정열, 신C",남희성
2,47497773,트레이스 - 신들의 시선,https://page.kakao.com/content/47497773,"액션, 퓨전, 먼치킨, 초능력",//dn-img-page.kakao.com/download/resource?kid=...,어느날 갑자기 세상에 나타나 인간들을 공격하기 시작한 정체불명의 존재 트러블.\n트...,All,화,네스티캣,네스티캣,NaN
3,47740585,나의 그녀는 구미호,https://page.kakao.com/content/47740585,판타지,//dn-img-page.kakao.com/download/resource?kid=...,어느날 나타난 사람을 빨아 먹는 나무요괴 미녀..\n그리고 양기가 충만한 주인공 봉...,Fifteen,수,프리,서락,NaN
4,47936014,신인왕좌,https://page.kakao.com/content/47936014,"판타지, 천재, 검사, 마법사, 격투가, 용병, 정령술사, 용사, 힐러",//dn-img-page.kakao.com/download/resource?kid=...,유명 판타지 소설을 만화화 한 작품!\n\n마족이 강세하고 인류는 멸종 위기에 직면...,All,토,Sariel,Sariel,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1493,68138491,내가 주인공인 줄 알았다,https://page.kakao.com/content/68138491,"로판, 로맨스판타지, 오해물, 빙의물, 능글남, 츤데레남, 집착남, 다정남, 왕족/...",//dn-img-page.kakao.com/download/resource?kid=...,로판 소설에 빙의했다.\n이름조차 등장하지 않는 엑스트라로.\n\n이게 바로 엑스트...,All,수,솔라,비로,서가린
1494,68144406,이리초파랑,https://page.kakao.com/content/68144406,"무협, 정통, 생존, 복수, 무림, 정파, 협객",//dn-img-page.kakao.com/download/resource?kid=...,"""보국구대문(保國九大門)의 영웅들이여---!!”\n\n멸국의 위기에 의기로 일어선\...",Fifteen,금,사마달,사마달,NaN
1495,68148725,붉은 달밤의 끝자락에서,https://page.kakao.com/content/68148725,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 운명...",//dn-img-page.kakao.com/download/resource?kid=...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",Fifteen,토,오공시,오공시,NaN
1496,68148783,붉은 달밤의 끝자락에서 [19세 완전판],https://page.kakao.com/content/68148783,"BL, 동양풍, 운명, 다정, 사랑꾼, 절대강, 집착, 소심, 아방, 소유욕, 질투...",//dn-img-page.kakao.com/download/resource?kid=...,"저주받은 아이라 손가락질 받으며 외롭게 살아온 자운.\n\n어느 날, 괴질이 온 마...",Nineteen,토,오공시,오공시,NaN
